In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Data Collection

### Files sources

In [10]:
# URLs of the files
train_datas_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_datas_url, 'module4_exercise_train.zip')
download_file(test_data_url, 'Neighborhood_Market_data.csv')

Downloaded module4_exercise_train.zip from https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip
Downloaded Neighborhood_Market_data.csv from https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv


#### CityMart

In [ ]:
# read "CityMart_data.csv"

#### Greenfield_Grocers

In [ ]:
# read "Greenfield_Grocers_data.csv"

#### Outlet_data

In [ ]:
# read "SuperSaver_Outlet_data.xlsx"

#### HighStreet_Bazaar

In [ ]:
# read 'HighStreet_Bazaar_data.json'

#### Aggregate

#### Simple baseline

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):

    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)

    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None

    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)

    # Return the average score
    return np.mean(scores)

In [ ]:
# get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### API sources

In [12]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {e}")
# password =
# prices =

In [ ]:
# df_prices =

#### Aggregate

In [ ]:
# data = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')
# data

In [ ]:
# get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Scrapping sources

In [13]:
# Full setup in one cell
!pip install selenium

# Install Chrome
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def create_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    return webdriver.Chrome(options=chrome_options)

# Usage example
driver = create_driver()

In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
# Note: Dans un environnement réel, il faudrait aussi définir `create_driver`
# et initialiser `driver = create_driver()` avant cette cellule.

# Set up the Selenium WebDriver (e.g., Chrome)
# driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
# driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
# NOTE: Le `driver` doit être initialisé et les installations de Chrome/ChromeDriver
# doivent être faites dans l'environnement du notebook pour que cette ligne fonctionne.
# driver.get(url)

# Wait for the page to fully load (increase time if needed)
# time.sleep(5)

# Get the fully rendered page source
# html = driver.page_source

# Pour que le code soit exécutable ici, nous devons simuler le driver, ou s'assurer
# que la variable `driver` est bien un objet WebDriver initialisé avant d'appeler get/page_source.
# Je suppose ici que vous avez déjà initialisé le `driver` dans une cellule précédente.

# --- Logique d'extraction (supposant que `html` est bien la source de la page) ---
# NOTE: Si vous exécutez ce code sans driver actif, il va échouer.
# Je vais me concentrer sur la logique d'extraction avec BeautifulSoup.

# Supposons que le code a été exécuté avec un driver fonctionnel:
# Parse the HTML with BeautifulSoup
# soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
# tables = soup.find_all('table')

# NOTE: Puisque je ne peux pas exécuter le driver, je vais simuler l'extraction
# en me concentrant sur la complétion du TODO, en m'appuyant sur la structure du TP.

# Scrape the second table (Exercise Data)
# course_table = tables[1]

# Nous allons définir les en-têtes pour une extraction propre
headers = ['item_code', 'packaging_type', 'marketing_score']

# Logique complétée pour l'extraction :
# Utilisation d'une variable `course_table` simulée pour la structure
# Le tableau scrappé a généralement les colonnes dans cet ordre: (index), item_code, packaging_type, marketing_score
# Si le `cols[0]` est l'index, nous commençons à cols[1]. Si ce sont les colonnes de données, nous commençons à cols[0].
# Basé sur le TP, les <td> sont: item_code, packaging_type, marketing_score
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')

    # Vérification simple pour s'assurer que nous avons suffisamment de colonnes
    if len(cols) >= 3:
        row_data = {
            'item_code': cols[0].text.strip(),
            'packaging_type': cols[1].text.strip(),
            'marketing_score': float(cols[2].text.strip()) # Conversion directe en float pour l'analyse
        }
        exercise_data.append(row_data)

# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data)

# Conversion de l'item_code en index
if 'item_code' in df_exercise.columns:
    df_exercise = df_exercise.set_index('item_code')

# Affichage du résultat
df_exercise.head()

,packaging_type,marketing_score
item_code,,
P0001,2,972.0
P0002,3,260.0
P0003,2,285.0
P0004,5,512.0
P0005,3,85.0


#### Aggregate

In [16]:
data = pd.merge(data, df_exercise, left_index=True, right_index=True, how='left')

In [17]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

np.float64(18.6498946573508)

### Generating Submission File

In [19]:
# X_test =  read  Neighborhood_Market_data

# read
df_StoreN =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_StoreN

,store_name,mass,dimension_length,dimension_width,dimension_height,days_since_last_purchase,package_volume,stock_age,last_modified
item_code,,,,,,,,,
P0002,Neighborhood_Market,5.51,51.79,46.72,72.02,344,174261.666176,287,2023-01-02
P0004,Neighborhood_Market,3.97,84.63,39.42,42.46,189,141651.425916,387,2023-01-04
P0005,Neighborhood_Market,5.99,39.33,83.51,5.12,183,16816.375296,382,2023-01-05
P0010,Neighborhood_Market,4.10,77.43,49.56,74.41,208,285543.225828,656,2023-01-10
P0013,Neighborhood_Market,6.96,95.39,34.61,23.24,114,76725.649196,755,2023-01-13
...,...,...,...,...,...,...,...,...,...
P1972,Neighborhood_Market,5.68,17.89,71.87,95.67,296,123008.113881,411,2028-05-25
P1977,Neighborhood_Market,1.53,84.32,64.91,66.30,171,362873.902560,702,2028-05-30
P1991,Neighborhood_Market,7.79,84.46,82.96,52.14,50,365334.635424,190,2028-06-13


In [20]:
df_StoreN = pd.merge(df_StoreN, df_prices, left_index=True, right_index=True, how='left')
df_StoreN = pd.merge(df_StoreN, df_exercise, left_index=True, right_index=True, how='left')

In [21]:
_, x_pred = get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold', X_data_test = df_StoreN)

In [23]:
submission = pd.DataFrame({
    'item_code': df_StoreN.index,
    'quantity_sold': 0 # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()

  item_code  quantity_sold
0     P0002              0
1     P0004              0
2     P0005              0
3     P0010              0
4     P0013              0